In [17]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [18]:
# Reading the datasets
payment_info = pd.read_csv(r"C:\Users\Hitheswar.Reddy\Downloads\payment_information.csv")
subscription_info = pd.read_csv(r"C:\Users\Hitheswar.Reddy\Downloads\subscription_information.csv")
industry_client_details = pd.read_csv(r"C:\Users\Hitheswar.Reddy\Downloads\industry_client_details.csv")
financial_info = pd.read_csv(r"C:\Users\Hitheswar.Reddy\Downloads\finanical_information.csv")

In [19]:
payment_info.head()

,client_id,payment_date,amount_paid,payment_method
0,6292156167,9/16/2019,447.0,Bank Transfer
1,7462725203,5/21/2018,379.7,Bank Transfer
2,4698004907,9/11/2021,435.1,Check
3,3510240337,12/7/2020,413.1,Check
4,7501599785,3/4/2019,61.1,Bank Transfer


In [20]:
subscription_info.head()

,client_id,subscription_type,start_date,end_date,renewed
0,1131383004,Yearly,2020-11-11,2021-11-11,False
1,4309371709,Monthly,2021-05-24,2021-06-23,True
2,3183675157,Yearly,2021-12-25,2022-12-25,True
3,5371694837,Monthly,2020-03-14,2020-04-13,True
4,5157113076,Monthly,2019-11-07,2019-12-07,False


In [21]:
financial_info.head()

,Unnamed: 0,start_date,end_date,inflation_rate,gdp_growth_rate
0,0,2018-01-01,2018-03-31,5.77,3.51
1,1,2018-04-01,2018-06-30,1.17,2.15
2,2,2018-07-01,2018-09-30,1.56,1.82
3,3,2018-10-01,2018-12-31,2.78,2.43
4,4,2019-01-01,2019-03-31,6.91,3.44


In [22]:
industry_client_details.head()

,client_id,company_size,industry,location
0,4280387012,Large,Finance Lending,Mumbai
1,2095513148,Small,Finance Lending,Chennai
2,7225516707,Medium,Finance Lending,New Delhi
3,8093537819,Large,Block Chain,Mumbai
4,4387541014,Medium,Hyper Local,Banglore


In [23]:
# Coverting all date columns to datetime format
payment_info["payment_date"] = pd.to_datetime(payment_info["payment_date"])
subscription_info["start_date"] = pd.to_datetime(subscription_info["start_date"])
subscription_info["end_date"] = pd.to_datetime(subscription_info["end_date"])
financial_info["start_date"] = pd.to_datetime(financial_info["start_date"])
financial_info["end_date"] = pd.to_datetime(financial_info["end_date"])

# Question 1: How many finance lending and blockchain clients does the organization have?

In [24]:
finance_lending_clients = industry_client_details[industry_client_details['industry'] == 'Finance Lending']['client_id'].nunique()
blockchain_clients = industry_client_details[industry_client_details['industry'] == 'Block Chain']['client_id'].nunique()
finance_and_blockchain_clients = industry_client_details[industry_client_details['industry'].isin(['Block Chain','Finance Lending'])]['client_id'].nunique()
print(f"Number of Finance Lending clients: {finance_lending_clients}")
print(f"Number of Blockchain clients: {blockchain_clients}")
print(f"Number of Blockchain and Finance Lending clients Combined: {blockchain_clients+finance_lending_clients}")

Number of Finance Lending clients: 22
Number of Blockchain clients: 25
Number of Blockchain and Finance Lending clients Combined: 47


# Question 2: Which industry in the organization has the highest renewal rate?

In [25]:
# Merge subscription_info with industry_client_details to get industry information
merged_subscription_industry = pd.merge(subscription_info, industry_client_details, on='client_id', how='outer')
print(subscription_info.shape)
print(industry_client_details.shape)
print(merged_subscription_industry.shape)

(100, 5)
(100, 4)
(100, 8)


In [26]:
merged_subscription_industry.head()

,client_id,subscription_type,start_date,end_date,renewed,company_size,industry,location
0,1131383004,Yearly,2020-11-11,2021-11-11,False,Large,Finance Lending,Banglore
1,4309371709,Monthly,2021-05-24,2021-06-23,True,Large,Hyper Local,Hyderabad
2,3183675157,Yearly,2021-12-25,2022-12-25,True,Small,Gaming,Chennai
3,5371694837,Monthly,2020-03-14,2020-04-13,True,Large,AI,New Delhi
4,5157113076,Monthly,2019-11-07,2019-12-07,False,Medium,Gaming,Banglore


## since the shape is same after join we can say both dataframes are unique on ClientId level so we can directly do a group by on industry and take the mean of the renewed column

In [27]:
renewal_rate_by_industry = merged_subscription_industry.groupby('industry')['renewed'].mean().sort_values(ascending=False)
renewal_rate_by_industry

industry
Gaming             0.727273
AI                 0.636364
Finance Lending    0.545455
Hyper Local        0.450000
Block Chain        0.440000
Name: renewed, dtype: float64

In [28]:
highest_renewal_industry = renewal_rate_by_industry.idxmax()
highest_renewal_rate = renewal_rate_by_industry.max()
print(f"The industry with the highest renewal rate is {highest_renewal_industry} with a renewal rate of {highest_renewal_rate:.2%}")

The industry with the highest renewal rate is Gaming with a renewal rate of 72.73%


# Question 3: What was the average inflation rate when their subscriptions were renewed?

In [29]:
# Filtering only renewed subscriptions
renewed_subs = subscription_info[subscription_info["renewed"] == True]

#### Assuming right after the end date of their subscription, the new subscription gets renewed so taking the end date as new renewal date

In [30]:
# Creating a function for matching the renewal date with inflation date ranges
def get_inflation_rate(renewal_date):
    for _, row in financial_info.iterrows():
        if row["start_date"] <= renewal_date <= row["end_date"]:
            return row["inflation_rate"]
    return None

renewed_subs["renewal_inflation"] = renewed_subs["end_date"].apply(get_inflation_rate)
average_inflation_at_renewal = renewed_subs["renewal_inflation"].dropna().mean()
print(f"The average inflation rate is {average_inflation_at_renewal}")

The average inflation rate is 4.3118


# Question 4: What is the median amount paid each year for all payment methods?

#### Assuming the question is to calculate the median amount of each year for all payments combined

In [31]:
# Extract year from payment_date
payment_info['year'] = payment_info['payment_date'].dt.year
# Calculate median amount paid each year
median_amount_paid_per_year = payment_info.groupby('year')['amount_paid'].median()
print("Median amount paid each year:")
print(median_amount_paid_per_year)

Median amount paid each year:
year
2018    235.7
2019    360.9
2020    284.5
2021    306.8
2022    288.0
Name: amount_paid, dtype: float64
